# Stress Prediction using ANN

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
path = '../../../data/combined_subjects.csv'
data = pd.read_csv(path, index_col=0)
data.head()

In [ ]:
pd.set_option('display.max_rows', 10)

### Data Preparation

#### Merged amusement
First we will merge the amusement data with the baseline data as after the EDA we found out that they are very simmilar.

In [ ]:
data_merged_am = data.copy()
# baseline = 0
data_merged_am["label"] = data_merged_am["label"].replace([1], 0)

# stressed = 1
data_merged_am["label"] = data_merged_am["label"].replace([2], 1)

data_merged_am["label"].unique()

Now we will split the data into train, validation and test sets.

In [ ]:
features = ['net_acc_std', 'net_acc_max', 'EDA_tonic_mean', 'EDA_tonic_min', 'EDA_tonic_max']
X = data_merged_am[features]
y = data_merged_am["label"]

X_train_merged_am, X_test_merged_am, y_train_merged_am, y_test_merged_am = train_test_split(X, y, test_size=0.2, random_state=42) # 80% training (1337 samples) and 20% test (334 samples)
X_val_merged_am, X_test_merged_am, y_val_merged_am, y_test_merged_am = train_test_split(X_test_merged_am, y_test_merged_am, test_size=0.1, random_state=42) # 90% of test set is used for validation (301 samples) and 10% for testing (33 samples)

We also need to scale the data so it lies between 0 and 1. This is important because the NN algorithm works better with scaled data, as generally activation function use values between 0 and 1.

In [ ]:
scaler = MinMaxScaler()
X_train_merged_am[features] = scaler.fit_transform(X_train_merged_am[features])
X_val_merged_am[features] = scaler.transform(X_val_merged_am[features])
X_test_merged_am[features] = scaler.transform(X_test_merged_am[features])

#### Dropped amusement
First we will drop the amusement data to see if merging it with baseline confuses the algorithm.

In [ ]:
data_no_am = data.copy()

# baseline = 0
data_no_am["label"] = data_no_am["label"].replace([1], 0)

# stressed = 1
data_no_am["label"] = data_no_am["label"].replace([2], 1)

data_merged_am["label"].unique()

Now we will again split the data into train, validation and test sets and scale it.

In [ ]:
X = data_no_am[features]
y = data_no_am["label"]

X_train_no_am, X_test_no_am, y_train_no_am, y_test_no_am = train_test_split(X, y, test_size=0.2, random_state=42) # 80% training and 20% test
X_val_no_am, X_test_no_am, y_val_no_am, y_test_no_am = train_test_split(X_test_no_am, y_test_no_am, test_size=0.1, random_state=42) # 90% of test set is used for validationand 10% for testing

scaler = MinMaxScaler()
X_train_no_am[features] = scaler.fit_transform(X_train_no_am[features])
X_val_no_am[features] = scaler.transform(X_val_no_am[features])
X_test_no_am[features] = scaler.transform(X_test_no_am[features])

### Modeling and Training

We will create a function that will create, compile and train a model so we can easily try different models and compare them. We will use the Sequential model from Keras, which is a linear stack of layers. We will add the Dense layers, which are just regular densely connected NN layers. The last layer will have 3 neurons (the number of labels) by default. We will use the relu activation function for the hidden layers and the softmax activation function for the last one. The softmax function is used for multiclass classification problems, it returns the probability of each class.

In [ ]:
EPOCHS = 100
def build_model(neurons_per_layer=[64, 64], n_outputs=2):
    model = Sequential()

    for i in range(len(neurons_per_layer)):
            model.add(Dense(neurons_per_layer[i], activation='relu'))

    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

#### Merged amusement
In order to decide for how many epochs we will train the model, we will devide the lenght of the dataset by the batch size and multiply it by two. This will give us a good estimate of how many epochs we need to train the model.

In [ ]:
batch_size = 32
len(y_train_merged_am) / batch_size * 2

The first model we will create will behave two layers with 64 neurons each. We will train for 20 epochs and use a standard batch size of 32.

In [ ]:
model1 = build_model()
history1 = model1.fit(x=X_train_merged_am, y=y_train_merged_am, epochs=EPOCHS, validation_data=(X_val_merged_am, y_val_merged_am))

We will also try models with 2 layers of 128 nodes and 3 layers with 1 layer of 64 and 2 layers of 128 nodes. We will again train for 20 epochs and use a standard batch size of 32.

In [ ]:
model2 = build_model([128, 128])
history2 = model2.fit(x=X_train_merged_am, y=y_train_merged_am, epochs=EPOCHS, validation_data=(X_val_merged_am, y_val_merged_am))

In [ ]:
model3 = build_model([512, 256, 256])
history3 = model3.fit(x=X_train_merged_am, y=y_train_merged_am, epochs=EPOCHS, validation_data=(X_val_merged_am, y_val_merged_am))

In [ ]:
model3.save("ann_merged_amusement_top_5_feat")

#### Dropped amusement

We will train the same models as the ones we trained with the merged data.

In [ ]:
batch_size = 32
len(y_train_no_am) / batch_size * 2

In [ ]:
model1_no_am = build_model()
history1_no_am = model1_no_am.fit(x=X_train_no_am, y=y_train_no_am, epochs=EPOCHS, validation_data=(X_val_no_am, y_val_no_am))

In [ ]:
model2_no_am = build_model([128, 128])
history2_no_am = model2_no_am.fit(x=X_train_no_am, y=y_train_no_am, epochs=EPOCHS, validation_data=(X_val_no_am, y_val_no_am))

In [ ]:
model3_no_am = build_model([512, 256, 256])
history3_no_am = model3_no_am.fit(x=X_train_no_am, y=y_train_no_am, epochs=EPOCHS, validation_data=(X_val_no_am, y_val_no_am))

In [ ]:
model3_no_am.save("ann_no_amusement_top_5_feat")

### Evaluation 
In order to compare the three models we will plot the loss and accuracy of all of them.

In [ ]:
def plot_evaluation(eval_type='accuracy', histories=[], labels=[]):
    fig, axs = plt.subplots(ncols=2, figsize=(16,5))
    for i in range(len(histories)):
        axs[0].plot(histories[i].history[eval_type])
        axs[1].plot(histories[i].history['val_' + eval_type])

    for ax in axs.flat:
        ax.set(xlabel='Epoch', ylabel=eval_type)
        ax.legend(labels, loc='upper left')

    fig.suptitle(f'Model train and validation {eval_type}')

#### Merged amusement

In [ ]:
results = [history1, history2, history3]
labels = ['model-64,64', 'mode-128,128', 'model-512,128,128']
plot_evaluation('accuracy', results, labels)

In [ ]:
plot_evaluation('loss', results, labels)

We can conclude that the model with 3 layers of 512, 128 and 128 nodes in each layer is the best model. It has a loss of ~0.15 and accuracy of ~96%. We can also see that the model is not overfitting as the loss and accuracy of the validation set are very close to the ones of the training set. Now we will plot a confusion matrix to see how the model performs on each class.

In [ ]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

y_pred_merged_am = model3.predict(X_val_merged_am)
cm = confusion_matrix(y_val_merged_am, y_pred_merged_am.argmax(axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

In [ ]:
print(f"accuracy_score: {round(accuracy_score(y_val_merged_am, y_pred_merged_am.argmax(axis=1)), 3)}")
print(f"balanced_accuracy: {round(balanced_accuracy_score(y_val_merged_am, y_pred_merged_am.argmax(axis=1)), 3)}")
print(f"f1_score: {round(f1_score(y_val_merged_am, y_pred_merged_am.argmax(axis=1)), 3)}")
print(f"recall_score: {round(recall_score(y_val_merged_am, y_pred_merged_am.argmax(axis=1)), 3)}")
print(f"precision_score: {round(precision_score(y_val_merged_am, y_pred_merged_am.argmax(axis=1)), 3)}")

We can see that less than 5% of the data is misclassified. Additionally, the model has way higher precission than recall, most likely because of the fact that we have more data for the baseline class than the stress class.

#### Dropped amusement

In [ ]:
results_ = [history1_no_am, history2_no_am, history3_no_am]
labels_ = ['model-64,64', 'mode-128,128', 'model-512,128,128']
plot_evaluation('accuracy', results_, labels_)

In [ ]:
plot_evaluation('loss', results_, labels_)

We can conclude that the model with 3 layers of 512, 128 and 128 nodes in each layer is the best model. It has a loss of ~0.15 and accuracy of ~94%. We can also see that the model is not overfitting as the loss and accuracy of the validation set are very close to the ones of the training set. Now we will again plot a confusion matrix to see how the model performs on each class.

In [ ]:
y_pred_no_am = model3_no_am.predict(X_val_no_am)
cm = confusion_matrix(y_val_no_am, y_pred_no_am.argmax(axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

In [ ]:
print(f"accuracy_score: {round(accuracy_score(y_val_no_am, y_pred_no_am.argmax(axis=1)), 3)}")
print(f"balanced_accuracy: {round(balanced_accuracy_score(y_val_no_am, y_pred_no_am.argmax(axis=1)), 3)}")
print(f"f1_score: {round(f1_score(y_val_no_am, y_pred_no_am.argmax(axis=1)), 3)}")
print(f"recall_score: {round(recall_score(y_val_no_am, y_pred_no_am.argmax(axis=1)), 3)}")
print(f"precision_score: {round(precision_score(y_val_no_am, y_pred_no_am.argmax(axis=1)), 3)}")

We can see that less than 5% of the data is misclassified. Additionally, in contrast with the `merged amusement` model, the `no amusement` madel has higher recall than precission. The reason for the slightly worse performance is most likely the smaller  amount of data.

#### Explainability

In order to get a better understanding of how the model works we will use the SHAP library. SHAP is a game theoretic approach to explain the output of any machine learning model. SHAP values represent a feature's responsibility for a change in the model output. The sum of the SHAP values equals the difference between the expected model output and the model output when all features are set to their average value.

In [ ]:
from explainerdashboard import ClassifierExplainer, ExplainerDashboard

In [ ]:
import types
def predict_proba(self, X):
    pred = self.predict(X).argmax(axis=1)
    return np.array([1-pred, pred]).T
model3.predict_proba = types.MethodType(predict_proba, model3)
model3_no_am.predict_proba = types.MethodType(predict_proba, model3_no_am)

#### Merged amusement

In [ ]:
explainer_merged_am = ClassifierExplainer(model3, X_test_merged_am, y_test_merged_am)
ExplainerDashboard(explainer_merged_am, mode="inline").run(8765)

#### Dropped amusement

In [ ]:
explainer_no_am = ClassifierExplainer(model3_no_am, X_test_no_am, y_test_no_am)
ExplainerDashboard(explainer_no_am, mode="inline").run(8766)